### ***如果你是某P2P租车的技术负责人，你会如何设计个性化推荐和搜索排序***
- 阐述相似车型，搜索排序的设计方法
- 可能的embedding策略

#### ***车型相似度***
对车型的相似计算，可以通过将车型Embedding之后，然后计算两车型Embedding后向量的余弦值的大小来评估两车型的相似度。
具体的Embedding方法，我们可以采用List Embedding, 顾名思义就是通过学习某个车型在一个队列中的上下文的信息，即与在用户
对车型的行为list里面，与当前车型位置越近的相似度越高。

![](images/list.png)

<p align="center">***Figure 1*** - List Embedding</p>

#### ***Embedding策略***
Embedding策略可采用类似于word2vec中的skip-gram的方法。从候选的点击list中构建与当前item左右相邻的m个item以及
最终选定并预定的车型作为正样本，而负样本的选取则从没有选中的item中随机采样得到,见上面的List Embedding示意图。

对于P2P租车业务，有以下几个特征：
1. 对于大多数用户来说P2P租车并不是高频行为，很多用户可能一年才使用一次租车服务。
2. 相比于点击浏览数据用户达成租车交易的数据就更加稀疏。
3. 由于使用的频率低，时间跨度长，用户的喜欢偏好会发生改变。

针对单个用户数据稀疏，实际交易数据稀少，以及偏好改变的应用特征。Embedding 策略可以从以下两个方向改进：
1. 将细粒度数据，聚合成粗粒数据；从对单个用户，单个行为List的Embedding，变为一类用户, 一类操作List的Embedding。
    典型的Embedding方法有User Type Embedding和List Type Embedding 分别用基础属性将用户和List分桶聚合来做Embedding。
    这样Embedding策略很好的解决的数据稀疏，和用户类型发生改变的问题。
    
2. 此外，在线收集用户近期点击过的List，浏览过的List, 跳过的List，用户的Wish List，用户联系单没有预定，用户两周内用户预定的List。
    用这些List训练出对商品不同维度的Embedding,可以弥补交易数据稀疏的不足，同时从不同的维度理解用户对商品的行为。
    收集实时的数据进行训练，可以获取用户近期的喜好，来解决用户喜好发生变化的问题。

此外在构建Embedding训练的损失函数的时候，将车主拒绝的item作为负样本考虑进去，在推荐时候可以过滤掉类似的车型，减少拒单的概率。

![](images/type.png)

<p align="center">***Figure2*** - Type Embeddings</p>

#### ***搜索排序***
可以采用排序预测模型比如RankNet, ListNet，LambdaNet等，基于listing， User ，query，cross, 和 Embedding 
特征对排序进行预测。相比于对单个物品进行评分的模型比如经典的GBDT模型，lambdaNet等模型可以很好得考虑到pairwise的顺序问题。